## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/data/zebrafish_pair1.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([1., 1., 1.])
mov_spacing = np.array([1., 1., 1.])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Alignment
---

### affine

In [2]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':6.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(8.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':1200,
    },
}

affine_kwargs = {
    'initial_condition':'CENTER',
}

steps = [('affine', {**common_kwargs, **affine_kwargs},),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

LEVEL:  0  ITERATION:  0  METRIC:  -0.4461030012717924
LEVEL:  0  ITERATION:  1  METRIC:  -0.4465659281266132
LEVEL:  0  ITERATION:  2  METRIC:  -0.4469774785685831
LEVEL:  0  ITERATION:  3  METRIC:  -0.4473408431766365
LEVEL:  0  ITERATION:  4  METRIC:  -0.447659346581875
LEVEL:  0  ITERATION:  5  METRIC:  -0.4479377865705412
LEVEL:  0  ITERATION:  6  METRIC:  -0.44817886857032924
LEVEL:  0  ITERATION:  7  METRIC:  -0.44838554883226256
LEVEL:  0  ITERATION:  8  METRIC:  -0.4485572132169357
LEVEL:  0  ITERATION:  9  METRIC:  -0.4486900168898224
LEVEL:  0  ITERATION:  10  METRIC:  -0.44881233394087056
LEVEL:  0  ITERATION:  11  METRIC:  -0.44902799565297696
LEVEL:  0  ITERATION:  12  METRIC:  -0.44931549220399836
LEVEL:  0  ITERATION:  13  METRIC:  -0.44959911413872566
LEVEL:  0  ITERATION:  14  METRIC:  -0.4498501019709971
LEVEL:  0  ITERATION:  15  METRIC:  -0.4500767030900825
LEVEL:  0  ITERATION:  16  METRIC:  -0.4502744062139313
LEVEL:  0  ITERATION:  17  METRIC:  -0.45043227338094